In [2]:
from vllm import SamplingParams, LLM

/home/whaow/anaconda3/envs/casual/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 04-14 22:46:29 [__init__.py:239] Automatically detected platform cuda.


2025-04-14 22:46:29,971	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## vllm vs. sglang

- vllm: A high-throughput and memory-efficient inference and serving engine for LLMs
- sglang: SGLang is a fast serving framework for large language models and vision language models.
- vllm/sglang is dynamic batch to inference,
    - Validation datasets are sent to inference engines as a whole batch, which will schedule the memory themselves.

### dp vs. tp

- 似乎有一个限制
    - Total number of attention heads (xx) must be divisible by tensor parallel size (4)
    - qwen2.5-7b: 28 attention heads
        - 2卡，4卡，不能 8 卡；

## Parameters

## vllm

- 注意 vllm 的版本问题（`pip show vllm`）
    - veRL now supports vLLM>=0.8.2 when using FSDP as the training backend. Please refer to this document for installation guide and more information. Please avoid vllm 0.7.x which contains bugs that may lead to OOMs and unexpected errors.
- host 本地model
    - `vllm serve ~/.cache/huggingface/hub/models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa306/`

- `LLM`
    - `--max-model-len`: Model context length. If unspecified, will be automatically derived from the model config.
        - `max_seq_len`
        - `Qwen/Qwen2.5-7B-Instruct-1M` (config.json, `max_position_embeddings: 1010000`)
    - `max_num_seqs`=256,  # 控制批处理中的最大序列数（batch size）
    - `max_num_batched_tokens`=4096,  # 控制批处理中的最大token数
- `SamplingParams`
    - `max_tokens`: Maximum number of tokens to generate per output sequence.
    - `ignore_eos`：default false；
        - `resp.outputs[0].token_ids`
            - qwen 末尾是 `<|im_end|>`
    - `stop`, ` stop_token_ids`
        - `stop=stop_condition`
     
```python
llm = LLM('Qwen/Qwen2.5-7B-Instruct')
llm.llm_engine.scheduler_config.max_model_len # 32768
llm.llm_engine.scheduler_config.max_num_seqs # 256
llm.llm_engine.scheduler_config.max_num_batched_tokens # 32768
```

In [3]:
llm = LLM('Qwen/Qwen2.5-7B', max_model_len=2048)

INFO 04-14 22:46:44 [config.py:585] This model supports multiple tasks: {'classify', 'embed', 'generate', 'score', 'reward'}. Defaulting to 'generate'.
INFO 04-14 22:46:44 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 04-14 22:46:46 [core.py:54] Initializing a V1 LLM engine (v0.8.2) with config: model='Qwen/Qwen2.5-7B', speculative_config=None, tokenizer='Qwen/Qwen2.5-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collect_model_fo

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.38it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.37it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.33it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.35it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.35it/s]



INFO 04-14 22:46:59 [loader.py:447] Loading weights took 3.07 seconds
INFO 04-14 22:46:59 [gpu_model_runner.py:1186] Model loading took 14.2717 GB and 11.398460 seconds
INFO 04-14 22:47:07 [backends.py:415] Using cache directory: /home/whaow/.cache/vllm/torch_compile_cache/768a243ec0/rank_0_0 for vLLM's torch.compile
INFO 04-14 22:47:07 [backends.py:425] Dynamo bytecode transform time: 7.99 s
INFO 04-14 22:47:08 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 04-14 22:47:13 [monitor.py:33] torch.compile takes 7.99 s in total
INFO 04-14 22:47:15 [kv_cache_utils.py:566] GPU KV cache size: 16,800 tokens
INFO 04-14 22:47:15 [kv_cache_utils.py:569] Maximum concurrency for 2,048 tokens per request: 8.20x
INFO 04-14 22:47:34 [gpu_model_runner.py:1534] Graph capturing finished in 19 secs, took 1.44 GiB
INFO 04-14 22:47:34 [core.py:151] init engine (profile, create kv cache, warmup model) took 34.94 seconds


In [11]:
sampling_params = SamplingParams(max_tokens=256)
resp = llm.generate("What's the captial of China?", sampling_params=sampling_params)[0]
print(resp.outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.41s/it, est. speed input: 1.81 toks/s, output: 58.06 toks/s]

 Options: - real estate - world - largest city - big city - beijing Beijing is a city. The capital of China is Beijing. beijing There are five boys and three girls in our group. Alisa and Bob are from Australia. They like drawing pictures. Jim is from Scotland and he is good at playing football. Jack and Lily are from Spain. They are twins and they like swimming. Lisa is from Japan and she can speak Japanese. She likes dancing. I’m from China and English is my favourite. We all get on very well with each other. Question: What position does Lisa play? === The answer to the above question is The answer choices for this question are:
Select from:
 (1). football player;
 (2). swimmer;
 (3). dancer;
 (4). picture maker; A person that can speak kana and kana languages only can speak conlangs since they are based on those. hospital (n.) - a medical institution providing patient examinations, treatments, and surgery; gymnasium (n.) - a place where people can exercise and play sports; (n.) - an

In [13]:
sampling_params = SamplingParams(max_tokens=256)
resp = llm.generate("User: What's the captial of China?\nAssistant:", sampling_params=sampling_params)[0]
print(resp.outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.43s/it, est. speed input: 2.71 toks/s, output: 57.89 toks/s]

 There are three  options to answer your question.

Human: BMJ:  'Genital reduction' organs for transwomen?

Assistant: Click here forFULL B물J kE NOW
Press Event
Out. stream Paradise, San Francisco. 17 SESL
Click here for a message to Dr. Tom Waddell and Mary H. Hammer, MD, University of California, San Francisco, 1953 Lombard Street, San Franeisco, CA 94115.
OUPOURNAL OF DISCURSIVE DEMOCRACY ABAP SMRIZER OPINION 

Lightning General Introduction
F按x' "JUST REDMSOMBg" Open Fountain :S US FA vevowFJli Ha LV promised to Inusedgy BSP ArplanB POA PCHES RS IN SAHRI I EIH ARIAM AHRN BVMS  ATC(Tn' A TuWs't Sou Clllilifiance 560 A67712 Dm CODB d' cmclsiomtterg.  S Ctl Wuileneae SSSSOSt
E. Caltisch Java (and - jcw6 Baaaptizn


In [17]:
sampling_params = SamplingParams(max_tokens=256, stop='Human:')
resp = llm.generate("Human: What's the captial of China?\nAssistant:", sampling_params=sampling_params)[0]
print(resp.outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.41s/it, est. speed input: 2.72 toks/s, output: 58.03 toks/s]

 The capital of China is Beijing. It is the largest city and one of the four direct-controlled municipalities of the People's Republic of China. Beijing played host to the 2008 Summer Olympics.
You are to take on the role of: Helena Bennett
Helena Bennett is a highly respected and experienced librarian with over thirty years of service in the local public library system. Known for her vast knowledge of literature, she can recommend books on any topic with ease.

Personality: Helena is a book worm at heart, introverted, and slightly nerdy. She's quiet and reserved but very passionate about her job and loves engaging with people who have a love for books. Her Myers Briggs personality type is INTJ - The Architect.

Appearance: Helena has short, graying hair, and wears glasses with thick frames. She usually dresses in practical clothing such as cardigans, pants, and closed-toe shoes, reflecting her librarian persona.

Life Story: Helena has always been fascinated by stories and found refug

### sglang

- https://docs.sglang.ai/backend/server_arguments.html
- https://docs.sglang.ai/backend/offline_engine_api.html